In [80]:
import os
import numpy as np
import pandas as pd
import geopandas
import matplotlib

import holoviews as hv
import shapely
# from holoviews.operation.datashader import aggregate

In [98]:
# TODO: Fetch CSV with location ID codes.
taxi_id = geopandas.read_file(os.path.join("assets", "taxi_zones", "taxi_zones.shp"))
print(taxi_id.head())

# taxi_id_shape.plot()

def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    if type(row[geom]) != shapely.geometry.multipolygon.MultiPolygon:
        exterior = row[geom].exterior

        if coord_type == 'x':
            # Get the x coordinates of the exterior
            return list( exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( exterior.coords.xy[1] )
    else:
        geoms_list = row[geom].geoms
        exterior_list = []
        
        for geom in geoms_list:
            exterior = geom.exterior
            
            if coord_type == 'x':
                exterior_list.append(list(exterior.coords.xy[0]))
            elif coord_type == 'y':
                exterior_list.append(list(exterior.coords.xy[1]))
        
        return exterior_list

taxi_id['x'] = taxi_id.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
taxi_id['y'] = taxi_id.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

# TODO: Find geofences for the given id codes.

# TODO: Overlay heatmap with geofences...

INFO:Fiona:Failed to auto identify EPSG: 7


   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  
0            EWR  POLYGON ((933100.9183527103 192536.0856972019,...  
1         Queens  (POLYGON ((1033269.243591294 172126.0078125, 1...  
2          Bronx  POLYGON ((1026308.769506663 256767.6975403726,...  
3      Manhattan  POLYGON ((992073.4667968601 203714.0759887695,...  
4  Staten Island  POLYGON ((935843.3104932606 144283.335850656, ...  


In [99]:
taxi_id.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,x,y
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,...","[933100.9183527103, 933091.0114800561, 933088....","[192536.08569720192, 192572.17526147654, 19260..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1...","[[1033269.2435912937, 1033439.6426391453, 1033...","[[172126.0078125, 170883.94580081105, 170808.2..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,...","[1026308.7695066631, 1026495.5934945047, 10265...","[256767.6975403726, 256638.61561836302, 256589..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,...","[992073.4667968601, 992068.6669922024, 992061....","[203714.07598876953, 203711.50219728053, 20371..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ...","[935843.3104932606, 936046.5648079664, 936387....","[144283.33585065603, 144173.41779872775, 14396..."


In [26]:
from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show

output_notebook()

NYC = x_range, y_range = ((-8242000,-8210000), (4965000,4990000))

plot_width  = int(750)
plot_height = int(plot_width//1.2)

def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    p.add_tools(BoxZoomTool(match_aspect=True))
    
    return p

options = dict(line_color=None, fill_color='blue', size=5)

from bokeh.tile_providers import STAMEN_TONER_BACKGROUND
p = base_plot()
p.add_tile(STAMEN_TONER_BACKGROUND)
p.multi_polygons(xs=[[[[1, 1, 2, 2]]]],
                 ys=[[[[3, 4, 4, 3]]]])
# p.add_tile(STAMEN_TONER_LABELS)
show(p)

Loading BokehJS ...